# Bases de données et création de procédés

Nous avons vu qu'une ACV consistait en deux grandes étapes : 
1. celle d'inventaire ;
2. puis celle de caractérisation d'inventaire.

En pratique, on s'appuie sur des bases de données (BDD) qui décrivent :
- la technosphère (ensemble des procédés et des flux qui les relient, dits flux intermédiaires) ; 
- la biosphère (ensemble des matières et substances potentiellement extraites ou émises par les procédés, dits flux élémentaires) ;
- les méthodes d'impacts (défintion des catégories d'impact, des indicateurs, des facteurs de caractérisation associés, et accessoirement des facteurs d'agrégation et/ou de normalisation)

SCHEMA INVENTAIRE IMPACTS FLUX

Commençons par les procédés. Nous utilisons couramment pour modéliser ces derniers la base de données ecoinvent qui est générique - elle recouvre plusieurs secteurs d'activités (Industrie, Énergie, Matériaux, Transports...).

Vous pouvez explorer cette BDD en ligne à ce lien : [https://ecoquery.ecoinvent.org/3.11/cutoff](https://ecoquery.ecoinvent.org/3.11/cutoff). (version, allocation, exemple de procédé)

Pour réaliser des ACV à partir de ces BDD, plusieurs logiciels sont disponibles (des libres tels OpenLCA et brightway ; des payants tels SimaPro, OneClick LCA ou Holis). Nous utilisons la librairie python brightway 2.5 dans ce qui suit car elle est open-source, permet de profiter de l'écosystème python, s'appuie explicitement sur le formalisme mathématique vu précédemment et évite l'effet "boîte noire".

**Documentation de brightway2.5**

Voir :

- documentation générale : https://docs.brightway.dev/en/latest/content/overview/index.html
- pense-bête : https://docs.brightway.dev/en/latest/content/cheatsheet/index.html

## Import des librairies et mise en place du projet

In [42]:
import bw2io as bi # ensemble des fonctions et classes pour importer et exporter (input/output)
import bw2data as bd # ... pour gérer les données du projet
import bw2calc as bc # ... pour faire des opérations
import bw2analyzer as ba # ... pour interpréter les résultats

import pandas as pd # pour utiliser un format de table pratique
import seaborn as sns # pour tracer des graphes à partir de ces tables
import matplotlib.pyplot as plt

In [3]:
bd.projects.set_current('2025-06-10_ecoinvent_311') # On se met dans un projet (ie un dossier) qui contient déjà ecoinvent 3.11 cut-off

In [4]:
bd.databases # On peut vérifier les BDD disponibles dans ce projet

Databases dictionary with 4 object(s):
	betons_armes
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff
	soletanche_database

## Exploration d'ecoinvent

On peut commencer par se poser les questions suivantes.

1. Combien y a-t-il d'éléments dans cette biosphère ?

In [5]:
biodb = bd.Database('ecoinvent-3.11-biosphere')
len(biodb)

9795

In [6]:
element = biodb.random() #On met dans une variable un élément de la biosphère choisi aléatoirement 
element

'Thiobencarb' (kilogram, None, ('water', 'surface water'))

In [7]:
element.as_dict()

{'categories': ('water', 'surface water'),
 'code': 'af2692ae-53c6-520d-b0cd-a1963ce0318a',
 'CAS number': '028249-77-6',
 'synonyms': [],
 'name': 'Thiobencarb',
 'database': 'ecoinvent-3.11-biosphere',
 'unit': 'kilogram',
 'type': 'emission',
 'id': 190810763435044976}

2. Combien y a-t-il de procédés dans cette technosphère ?

In [8]:
eidb = bd.Database('ecoinvent-3.11-cutoff') #On sélectionne la technosphere que l'on stocke dans une variable
len(eidb) #La longueur de celle-ci correspond au nombre de procédés différents disponibles

25412

In [9]:
activity = eidb.random()
activity

'sodium tripolyphosphate production' (kilogram, RER, None)

In [10]:
activity.as_dict()

{'comment': "This dataset represents the production of 1 kg of spray-dried powder of sodiumtripolyphosphate (STPP). STPP is a solid, inorganic compound present in the form of slightly hygroscopic granules. It is used in a large variety of household cleaning products, mainly as builder – but also in human foodstuff, animal feed, industrial cleaning processes and ceramics manufacture. Within the cleaning products STPP can be one of the ingredients of regular and compact laundry detergents, of automatic dishwashing detergents, of toilet cleaners or of surface cleaners. As builder in these detergents, STPP provides various functions – among them e.g. the sequestration of the water hardness, enabling surfactants to function effectively. Further functions of STPP are pH buffering, dirt emulsification or hydrolysis of grease. STPP used within the detergent industry is produced from sodium phosphate ortho-liquor. In this dataset, the material and energy inputs, as well as production of waste, 

3. Comment un procédé est-il relié aux autres ?

In [11]:
activity = eidb.search('anchovy') #On utilise une fonction de recherche par mot-clé, ici "anchovy"
activity # brightway retourne une liste de procédés

['landed anchovy by-catch, fresh to generic market for marine fish' (kilogram, GLO, None),
 'landed anchovy to generic market for marine fish' (kilogram, GLO, None),
 'market for landed anchovy, fresh' (kilogram, PE, None),
 'market for landed anchovy, fresh' (kilogram, RoW, None),
 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None),
 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None),
 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, RoW, None),
 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, RoW, None),
 'market for landed anchovy by-catch, fresh' (kilogram, GLO, None),
 'fishmeal and fish oil production, 63-65% protein, from fresh anchovy' (kilogram, RoW, None),
 'fishmeal and fish oil production, 63-65% protein, from fresh anchovy' (kilogram, PE, None),
 'market for fish oil, from anchovy' (kilogram, GLO, None),
 'anchovy, capture by steel purse seiner a

In [13]:
activity = eidb.search('anchovy, capture by steel purse seiner')[0] #On choisit le premier élément de la liste, en précisant le mot-clé
activity #Ici, activity sera bien un procédé, non une liste de procédés

'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)

On regarde maintenant quels sont ses échanges avec la technosphère et la biosphère.

In [14]:
list(activity.exchanges()) #Tous les échanges

[Exchange: 1.0 kilogram 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: -0.002032598343604353 kilogram 'market for antifouling paint emissions' (kilogram, GLO, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 17.88461398791684 megajoule 'market for diesel, burned in fishing vessel' (megajoule, GLO, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 0.006310948398860212 kilogram 'market for lubricating oil' (kilogram, RER, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 0.47813383803583204 kilogram day 'market for operation, reefer, freezing' (kilogram day, GLO, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 0.

In [15]:
list(activity.technosphere()) #Flux intermédiaires

[Exchange: -0.002032598343604353 kilogram 'market for antifouling paint emissions' (kilogram, GLO, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 17.88461398791684 megajoule 'market for diesel, burned in fishing vessel' (megajoule, GLO, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 0.006310948398860212 kilogram 'market for lubricating oil' (kilogram, RER, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 0.47813383803583204 kilogram day 'market for operation, reefer, freezing' (kilogram day, GLO, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 0.01301350223451024 kilogram 'market for purse seiner maintenance, steel' (kilogram, GLO, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 0.0130

In [16]:
list(activity.biosphere()) #Flux élémentaires

[Exchange: 0.0943237168289159 kilogram 'Discarded fish, pelagic, to ocean' (kilogram, None, ('water', 'ocean')) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>,
 Exchange: 1.3262161254194131 kilogram 'Fish, pelagic, in ocean' (kilogram, None, ('natural resource', 'biotic')) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>]

In [17]:
list(activity.production()) #Flux de référence (produit)

[Exchange: 1.0 kilogram 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None) to 'anchovy, capture by steel purse seiner and landing whole, fresh' (kilogram, ES, None)>]

N'oubliez pas la page pense-bête de brightway : https://docs.brightway.dev/en/latest/content/cheatsheet/inventory.html

Pour les personnes souhaitant en savoir plus sur la structure en graphe des données dans brightway : https://docs.brightway.dev/en/latest/content/overview/inventory.html 

Bien que nous adorions la tapenade, nous allons commencer par modéliser ce que tout élève des Ponts préfère : le BÉTON ARMÉ.

## Construction de procédés béton

Il en existe dans la base de données :

In [19]:
concrete = eidb.search('concrete')
concrete

['concrete, all types to generic market for concrete, medium strength' (cubic meter, CO, None),
 'concrete, all types to generic market for concrete, normal strength' (cubic meter, PE, None),
 'concrete, all types to generic market for concrete, normal strength' (cubic meter, ZA, None),
 'concrete, all types to generic market for concrete, medium strength' (cubic meter, PE, None),
 'concrete, all types to generic market for concrete, normal strength' (cubic meter, RNA, None),
 'concrete, all types to generic market for concrete, medium strength' (cubic meter, ZA, None),
 'concrete, all types to generic market for concrete, normal strength' (cubic meter, BR, None),
 'concrete, all types to generic market for concrete, medium strength' (cubic meter, RNA, None),
 'concrete, all types to generic market for concrete, normal strength' (cubic meter, CO, None),
 'concrete, all types to generic market for concrete, medium strength' (cubic meter, BR, None),
 'concrete, all types to generic marke

Existe-t-il des procédés modélisant des bétons spécifiquement français ?

Nous utilisons une compréhension de liste conditionnelle pour préciser la recherche :

In [20]:
concrete = [a for a in eidb if 'concrete' in a['name'].lower() and a['location'] == 'FR']
concrete

[]

Comme il n'y en a pas, nous allons modéliser deux formulations avec les données dont nous disposons, pour les comparer. On en profite pour prendre en compte des armatures (on étudie du béton armé) et de l'énergie (malaxage du béton).

Considérons les formulations suivantes pour 1 m3 de béton : 

- 350kg de ciment Portland (béton A) ou CEM III/B (béton B)
- 175kg d'eau
- 800kg de sable
- 1100kg de gravier

Pour le ferraillage, prenons un ratio de 100 kg d'acier par m3 de béton. Avez-vous une remarque ?

Pour l'énergie de malaxage, prenons l'hypothèse de 14.4 MJ par m3 de béton.

Les procédés mobilisés sont les suivants :

Béton | Flux | Procédé choisi dans ecoinvent | Région | Unité | Qté pour 1m3
:---: | :---: | :---: | :---: | :---: | :---:
Béton A | ciment Portland | market for cement, Portland | Europe without Switzerland | kg | 350
Béton B | CEM III/B | market for cement, CEM III/B | Europe without Switzerland | kg | 350
Béton A & B | eau | market for tap water | Europe without Switzerland | kg | 175
Béton A & B | sable | market for sand | Rest-of-World (RoW) | kg | 800 
Béton A & B | graviers | market for gravel, crushed | Rest-of-World (RoW) | kg | 1100
Béton A & B | ferraillage | market for reinforcing steel | Global | kg | 100
Béton A & B | énergie | diesel, burned in building machine | Global | MJ | 14.4

In [21]:
if 'betons_armes' in bd.databases : 
    print('procédés déjà importés !')
else : 
    imp = bi.ExcelImporter("/home/thibault.chevilliet@enpc.fr/Documents/Cours ACV notebook/Cours 2025/betons_armes.xlsx")
    imp.apply_strategies() #applique une série de routines sur la base de données, notamment une conversion des variables dans les bons formats
    imp.match_database(fields=('name', 'unit', 'location')) #identifie les échanges que les procédés de la base ont avec d'autres procédés de la même base
    imp.match_database('ecoinvent-3.11-cutoff', fields=('name', 'unit', 'location')) #identifie les échanges que les procédés de la base ont avec les procédés d'ecoinvent
    imp.statistics() #affiche les statistiques des "match" réalisés, notamment si certains procédés n'ont pas été trouvés ("unlinked exchanges") 
    imp.write_database() #écrit et sauvegarde la base de données

procédés déjà importés !


In [22]:
fgdb = bd.Database('betons_armes') #fgdb pour foreground database

In [23]:
beton_A = fgdb.search('Béton A')[0]
beton_A

'Béton A' (cubic meter, FR, ('custom',))

In [24]:
list(beton_A.edges())

[Exchange: 1 cubic meter 'Béton A' (cubic meter, FR, ('custom',)) to 'Béton A' (cubic meter, FR, ('custom',))>,
 Exchange: 350 kilogram 'market for cement, Portland' (kilogram, Europe without Switzerland, None) to 'Béton A' (cubic meter, FR, ('custom',))>,
 Exchange: 175 kilogram 'market for tap water' (kilogram, Europe without Switzerland, None) to 'Béton A' (cubic meter, FR, ('custom',))>,
 Exchange: 800 kilogram 'market for sand' (kilogram, RoW, None) to 'Béton A' (cubic meter, FR, ('custom',))>,
 Exchange: 1100 kilogram 'market for gravel, crushed' (kilogram, RoW, None) to 'Béton A' (cubic meter, FR, ('custom',))>,
 Exchange: 100 kilogram 'market for reinforcing steel' (kilogram, GLO, None) to 'Béton A' (cubic meter, FR, ('custom',))>,
 Exchange: 14.4 megajoule 'diesel, burned in building machine' (megajoule, GLO, None) to 'Béton A' (cubic meter, FR, ('custom',))>]

## Calcul des impacts environnementaux

Pour calculer des impacts environnementaux, il faut savoir quoi calculer, c'est-à-dire choisir une méthode d'impact. 

Remarque sur le mot méthode :
- Traditionnellement, une méthode d'impact désigne un ensemble de catégories d'impacts, chaque catégorie contenant des indicateurs environnementaux et les facteurs de caractérisation permettant de calculer ces derniers à partir des flux élémentaires (en réalité, pour chaque indicateur, à partir d'un sous-ensemble de la biosphère).
- dans brightway, une "method" désigne en fait un indicateur spécifique (calculé selon une certaine méthode d'impact), ce qui peut prêter à confusion.


Pour ce TD, nous choisissons la méthode d'impacts Environmental Footprint v3.1.

In [25]:
meth = [m for m in bd.methods if 'EF v3.1' in m[1] and 'no LT' not in m[1]]
meth

[('ecoinvent-3.11', 'EF v3.1', 'acidification', 'accumulated exceedance (AE)'),
 ('ecoinvent-3.11',
  'EF v3.1',
  'climate change',
  'global warming potential (GWP100)'),
 ('ecoinvent-3.11',
  'EF v3.1',
  'climate change: biogenic',
  'global warming potential (GWP100)'),
 ('ecoinvent-3.11',
  'EF v3.1',
  'climate change: fossil',
  'global warming potential (GWP100)'),
 ('ecoinvent-3.11',
  'EF v3.1',
  'climate change: land use and land use change',
  'global warming potential (GWP100)'),
 ('ecoinvent-3.11',
  'EF v3.1',
  'ecotoxicity: freshwater',
  'comparative toxic unit for ecosystems (CTUe)'),
 ('ecoinvent-3.11',
  'EF v3.1',
  'ecotoxicity: freshwater, inorganics',
  'comparative toxic unit for ecosystems (CTUe)'),
 ('ecoinvent-3.11',
  'EF v3.1',
  'ecotoxicity: freshwater, organics',
  'comparative toxic unit for ecosystems (CTUe)'),
 ('ecoinvent-3.11',
  'EF v3.1',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('ecoinvent-3.

### Calcul pour un indicateur et une demande

On s'intéresse dans un premier temps à l'impact sur le changement climatique en considérant le pouvoir de réchauffement global à 100 ans.

In [26]:
gwp100 = [ind for ind in meth if 'GWP100' in str(ind) and 'biogenic' not in str(ind) and 'fossil' not in str(ind) and 'land' not in str(ind)][0] # On sélectionne l'indicateur par compréhension de liste
gwp100

('ecoinvent-3.11',
 'EF v3.1',
 'climate change',
 'global warming potential (GWP100)')

On fait le calcul pour le béton A :

In [27]:
lca = bc.LCA(demand={beton_A : 1.0},method=gwp100) #Préparation de matrices réduites A, B, C
lca.lci()# Préparation du calcul de g. Si on appelle le score à cette étape, on obtient g.
lca.lcia() # Préparation du calcul de h.
lca.score # Ici le calcul se fait, et retourne le score d'impact

561.9650284777491

et maintenant pour le béton B :

In [29]:
beton_B = fgdb.search('Béton B')[0]
beton_B

'Béton B' (cubic meter, FR, ('custom',))

In [30]:
lca = bc.LCA(demand={beton_B : 1.0},method=gwp100)
lca.lci()
lca.lcia()
lca.score

391.69460121070745

### Calcul pour plusieurs indicateurs et plusieurs demandes

La logique est strictement la même que pour un indicateur et une demande. La fonction qui gère ce calcul réclame simplement que l'on range les arguments d'une manière spécifique, comme réalisé ci-dessous :

In [31]:
demands = { 'A' : {beton_A.id : 1.0}, 'B' : {beton_B.id : 1.0}} # Les demandes évaluée sont rangées dans un dictionnaires, elles ont un nom, ici nous choisissons A et B.

method_config = {'impact_categories' : meth} # les indicateurs d'impacts sont aussi fan
data_objs = bd.get_multilca_data_objs(demands, method_config)

lca = bc.MultiLCA(demands=demands, method_config= method_config, data_objs=data_objs)
lca.lci()
lca.lcia()
results = lca.scores

On range les résultats dans une table pour les voir et les tracer

In [67]:
df_results = pd.DataFrame.from_dict(results,orient='index')
res = pd.DataFrame(columns=['indicateur','beton','score'])
for r in df_results.index :
    res = pd.concat([pd.DataFrame([[str(r[0][2:3]),r[1],df_results.at[r,0]]], columns=res.columns), res], ignore_index=True)

/tmp/ipykernel_44114/1971927674.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([pd.DataFrame([[str(r[0][2:3]),r[1],df_results.at[r,0]]], columns=res.columns), res], ignore_index=True)


In [68]:
res# Visualition de la table de résultat

,indicateur,beton,score
0,"('water use',)",B,1.663520e+02
1,"('water use',)",A,1.761166e+02
2,('photochemical oxidant formation: human healt...,B,1.397576e+00
3,('photochemical oxidant formation: human healt...,A,1.637680e+00
4,"('particulate matter formation',)",B,2.990077e-05
5,"('particulate matter formation',)",A,2.872002e-05
6,"('ozone depletion',)",B,2.451686e-06
7,"('ozone depletion',)",A,2.760079e-06
8,"('material resources: metals/minerals',)",B,1.443926e-03
9,"('material resources: metals/minerals',)",A,1.784568e-03


On peut ajouter une colonne où l'on normalise par le maximum pour chaque indicateur

In [69]:
res['score_norm'] = [res.at[i,'score']/max(res[res.indicateur == res.at[i,'indicateur']]['score']) for i in res.index]
res

,indicateur,beton,score,score_norm
0,"('water use',)",B,1.663520e+02,0.944556
1,"('water use',)",A,1.761166e+02,1.000000
2,('photochemical oxidant formation: human healt...,B,1.397576e+00,0.853388
3,('photochemical oxidant formation: human healt...,A,1.637680e+00,1.000000
4,"('particulate matter formation',)",B,2.990077e-05,1.000000
5,"('particulate matter formation',)",A,2.872002e-05,0.960511
6,"('ozone depletion',)",B,2.451686e-06,0.888267
7,"('ozone depletion',)",A,2.760079e-06,1.000000
8,"('material resources: metals/minerals',)",B,1.443926e-03,0.809118
9,"('material resources: metals/minerals',)",A,1.784568e-03,1.000000


In [1]:
fig, ax = plt.subplots()
sns.catplot(data=res,x='indicateur',y='score_norm',kind='bar',hue='beton',errorbar=None)
plt.xticks(rotation=45, ha='right')

NameError: name 'plt' is not defined

In [95]:
import pandas as pd

df_impact =pd.read_excel('/home/thibault.chevilliet@enpc.fr/Documents/Cours ACV notebook/Cours 2025/impact_world_plus_2.1 (incl. CO2 uptake)_expert_version_ecoinvent_v311.xlsx')

In [ ]:
import Bw25_personal_functions as perso
indicators = perso.unique_list(p)
for row in pandas.index :



my_cf_data = [
    (biosphere_node_1, 42),
    (biosphere_node_1, 23, 'BR'),
    (biosphere_node_2, {
        'uncertainty_type': sa.TriangularUncertainty.id,
        'amount': 7,
        'loc': 7,
        'maximum': 21
    })
]
bd.Method(('<impact>', '<category>')).write(my_cf_data)

In [79]:
met = [m for m in bd.methods if 'water' in str(m)][0]
met

('ecoinvent-3.11',
 'CML v4.8 2016 no LT',
 'ecotoxicity: freshwater no LT',
 'freshwater aquatic ecotoxicity (FAETP inf) no LT')

In [ ]:
from polyviz import sankey

sankey(activity,met,level = 5)

Calculating supply chain score...
Sankey diagram generated.


('/home/thibault.chevilliet@enpc.fr/Documents/Cours ACV notebook/Cours 2025/anchovy capture by steel purse seiner and landing whole fresh kilogram ES ecoinvent311CML v48 2016 no LTecotoxicity freshwater no LTfreshwater aquatic ecotoxicity FAETP inf no LT sankey.html',
                                                source  \
 0   anchovy, capture by steel purse seiner and lan...   
 1                             activities below cutoff   
 2       m. for diesel, burned in fishing vessel (GLO)   
 3                        m. for lubricating oil (RER)   
 4        m. for purse seiner maintenance, steel (GLO)   
 ..                                                ...   
 62                m. for waste plastic, mixture (RoW)   
 63                    m. for waste polyethylene (RoW)   
 80                                          emissions   
 64                            activities below cutoff   
 81                                          emissions   
 
                                 

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/thibault.chevilliet@enpc.fr/snap/codium/common/.cache/gio-modules/libgiolibproxy.so
Gtk-Message: 14:42:45.356: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
Gtk-Message: 14:42:45.477: Failed to load module "canberra-gtk-module"
Gtk-Message: 14:42:45.480: Failed to load module "canberra-gtk-module"
